<a href="https://colab.research.google.com/github/hargurjeet/hpe/blob/main/Problem_2_hpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Importing all the requeired libraries

# For data preprocessing
import pandas as pd
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTENC

# To build visualizations
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats
import pylab

# Librareis for building pipelines
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# To build ML models
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/hargurjeet/hpe/main/Problem2.csv')
df.head()

,Y,V1,V2,V3,V4,V5,V6,V7,V8,V9
0,"9,111",0.0,0.0,0.343088,0.0,100.4014,100.6104,0.04,0.551271,3.361059
1,"10,918",0.0,0.0,0.343088,0.0,100.5697,100.7629,0.04,0.551271,3.361059
2,"13,402",0.0,0.0,0.343088,0.0,100.7521,100.8958,0.04,0.551271,3.361059
3,"11,530",0.0,0.0,0.357665,0.0,100.9278,101.0110,0.04,0.556536,3.559978
4,"9,972",0.0,0.0,0.357665,0.0,101.0532,101.1080,0.04,0.556536,3.559978
